# Jackknife correlation: 
###En algunas ocasiones se presenta dificultad para encontrar estimadores de un parámetro $\theta$ que posean propiedades deseables como insesgamiento y precisión; además, cuando se dispone de un estimador sesgado, generalmente no se dispone de metodología para la estimación del sesgo. Otras veces, se dispone de un estimador insesgado, pero no se conoce una expresión cerrada para su varianza o para un estimador de la misma, lo cual obliga a realizar aproximaciones de distinta naturaleza, las cuales no producen los resultados más deseables. 
#Descripción del método: 
###Descripción del método: 
#Ejemplo: 
###Un equipo de investigadores quiere estudiar si existe correlación en la presencia de dos sustancias (A y B) en el agua de los ríos. Para ello han realizado una serie de mediciones en las que se cuantifica la concentración de las dos sustancias en 10 muestras independientes de agua. Se sospecha que el instrumento de lectura sufre alguna avería que provoca que algunas lecturas se disparen, por esta razón se quiere emplear un método de correlación robusto. El objetivo de este ejemplo es ilustrar el método de Jackknife, por lo que se asume que se cumplen las condiciones para la correlación de Pearson. 
#Datos: 


In [ ]:
# Datos simulados de dos variables A y B
a = np.array([12,9,6,7,2,5,4,0,1,8])
b = np.array([3,5,1,9,5,3,7,2,10,5])

# Se introduce un outlier
a[5] = 20
b[5] = 16
# Gráfico
# ==============================================================================
fig, ax = plt.subplots(1, 1, figsize=(6,4))
ax.plot(a, label='A')
ax.plot(b, label='B')
ax.set_xlabel('ID muestra')
ax.set_ylabel('Concentración')
ax.set_title('Concentración sustancias A y B en las muestras')
x.legend();


# Jackknife correlation: 
###En algunas ocasiones se presenta dificultad para encontrar estimadores de un parámetro $\theta$ que posean propiedades deseables como insesgamiento y precisión; además, cuando se dispone de un estimador sesgado, generalmente no se dispone de metodología para la estimación del sesgo. Otras veces, se dispone de un estimador insesgado, pero no se conoce una expresión cerrada para su varianza o para un estimador de la misma, lo cual obliga a realizar aproximaciones de distinta naturaleza, las cuales no producen los resultados más deseables. 
#Descripción del método: 
###Descripción del método: 
#Ejemplo: 
###Un equipo de investigadores quiere estudiar si existe correlación en la presencia de dos sustancias (A y B) en el agua de los ríos. Para ello han realizado una serie de mediciones en las que se cuantifica la concentración de las dos sustancias en 10 muestras independientes de agua. Se sospecha que el instrumento de lectura sufre alguna avería que provoca que algunas lecturas se disparen, por esta razón se quiere emplear un método de correlación robusto. El objetivo de este ejemplo es ilustrar el método de Jackknife, por lo que se asume que se cumplen las condiciones para la correlación de Pearson. 
#Datos: 


In [ ]:
# Datos simulados de dos variables A y B
a = np.array([12,9,6,7,2,5,4,0,1,8])
b = np.array([3,5,1,9,5,3,7,2,10,5])

# Se introduce un outlier
a[5] = 20
b[5] = 16
# Gráfico
# ==============================================================================
fig, ax = plt.subplots(1, 1, figsize=(6,4))
ax.plot(a, label='A')
ax.plot(b, label='B')
ax.set_xlabel('ID muestra')
ax.set_ylabel('Concentración')
ax.set_title('Concentración sustancias A y B en las muestras')
x.legend();


##    b) Simular 5 sorteos con sustitución.


In [ ]:
import random
  m = m+1
print("")
print ("Veamos cuantos adivinaste xd")
print("")
print ("Adivinaste: ",reptar," Nada mal :y")
print("")
